In [1]:
import requests
import psycopg2
import socket
import json
from datetime import datetime
import os

In [5]:

# Substitui com os teus dados reais
# HOST = "db.nhbyslgfifrxsubejasw.supabase.co"
# # DB = "postgres"
# USER = "postgres"
# PASSWORD = "Database_99_!_route"  # Substitui por completo
# PORT = "5432"

HOST = os.environ['DB_HOST']
DB = os.environ['DB_NAME']
USER = os.environ['DB_USER']
PASSWORD = os.environ['DB_PASSWORD']
PORT = os.environ['DB_PORT']

try:
    print("🟡 A ligar ao Supabase...")
    connection = psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT,
        sslmode='require'
    )

    print("✅ Ligação estabelecida com sucesso!")

    cursor = connection.cursor()
    cursor.execute("SELECT NOW();")
    resultado = cursor.fetchone()
    print("🕒 Hora atual no servidor:", resultado)

    cursor.close()
    connection.close()
    print("🔒 Ligação fechada.")

except Exception as e:
    print("❌ Erro ao ligar à base de dados:", e)


🟡 A ligar ao Supabase...
✅ Ligação estabelecida com sucesso!
🕒 Hora atual no servidor: (datetime.datetime(2025, 5, 17, 22, 2, 20, 21948, tzinfo=datetime.timezone.utc),)
🔒 Ligação fechada.


In [9]:
#Connect to the server

print("🟡 A ligar ao Supabase...")
connection = psycopg2.connect(
    host=HOST,
    database=DB,
    user=USER,
    password=PASSWORD,
    port=PORT
)

print("✅ Ligação estabelecida com sucesso!")

cursor = connection.cursor()
cursor.execute("SELECT NOW();")
resultado = cursor.fetchone()
print("🕒 Hora atual no servidor:", resultado)


🟡 A ligar ao Supabase...
✅ Ligação estabelecida com sucesso!
🕒 Hora atual no servidor: (datetime.datetime(2025, 5, 17, 22, 3, 7, 76992, tzinfo=datetime.timezone.utc),)


In [10]:
API_KEY = os.environ['API_KEY']  # Replace with your actual API key
AIRPORT = 'LIS'

def get_flights(flight_type):
    url = f'https://aviation-edge.com/v2/public/timetable?key={API_KEY}&iataCode={AIRPORT}&type={flight_type}'
    response = requests.get(url)

    if response.status_code == 200:
        print(f"✅ {flight_type.capitalize()} flights fetched successfully.")
        return response.json()
    else:
        print(f"❌ Failed to fetch {flight_type} flights ({response.status_code}): {response.text}")
        return []

def save_flights(flights, flight_type):
    today = datetime.now().date()
    total = 0

    for flight in flights:
        try:
            cursor.execute('''
                INSERT INTO flights_2 (
                    flight_number, airline_iata, airline_name,
                    dep_iata, dep_time, arr_iata, arr_time,
                    reg_number, status, direction, fetch_date
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ''', (
                flight.get('flight', {}).get('iataNumber', ''),
                flight.get('airline', {}).get('iataCode', ''),
                flight.get('airline', {}).get('name', ''),
                flight.get('departure', {}).get('iataCode', ''),
                flight.get('departure', {}).get('scheduledTime', None),
                flight.get('arrival', {}).get('iataCode', ''),
                flight.get('arrival', {}).get('scheduledTime', None),
                flight.get('regNumber', ''),
                flight.get('status', ''),
                flight_type,
                today
            ))
            total += 1
        except Exception as e:
            print(f"⚠️ Error inserting flight {flight.get('flight', {}).get('iataNumber', 'UNKNOWN')}: {e}")
            connection.rollback()  # rolled back on error

    connection.commit()
    print(f"✅ {total} {flight_type} flights saved to the database.")


# 🚀 Fetch and store flights
departures = get_flights('departure')
arrivals = get_flights('arrival')

save_flights(departures, 'departure')
save_flights(arrivals, 'arrival')

cursor.close()
connection.close()
print("🔒 Database connection closed.")


✅ Departure flights fetched successfully.
✅ Arrival flights fetched successfully.
✅ 460 departure flights saved to the database.
✅ 511 arrival flights saved to the database.
🔒 Database connection closed.
